# Steps: 
1. Determine which variables to use
2. Cut down size of dataframe (maybe top 10 and do dummies? or an aggregate (# of pitbulls per month)
3. Create multivariate dataframe, including timestamp index
4. Scale, series_to_supervise
5. Fit data, split to train_test sets
6. Use univariate LSTM, tune to lowest MSE
7. Compare univariate to multivariate scores, make analysis
8. Save model and fit into function
9. Write new prediction function? 
10. Conclude, interpret
11. Make more visualizations
12. Blog about it
13. Where do we go from here? 
14. What is the "so what"? 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/raw_data.csv')

In [3]:
df.set_index('ValidDate', inplace = True)

In [4]:
df = df.drop(['ExpYear'], axis = 1)

In [5]:
df.head()

,LicenseType,Breed,Color,DogName,OwnerZip
ValidDate,,,,,
2014-12-02 09:40:53,Dog Individual Neutered Male,COCKAPOO,BROWN,CHARLEY,15236
2014-12-02 09:45:25,Dog Senior Citizen or Disability Neutered Male,GER SHEPHERD,BLACK/BROWN,TACODA,15238
2014-12-02 09:47:55,Dog Individual Spayed Female,GER SHEPHERD,BLACK,CHARLY,15205
2014-12-02 10:02:33,Dog Individual Spayed Female,LABRADOR RETRIEVER,BLACK,ABBEY,15143
2014-12-02 10:05:50,Dog Individual Female,GER SHORTHAIR POINT,BROWN,CHARLEY,15228


In [6]:
df['Breed'].value_counts().sum()

286724

In [7]:
#I would definitely need to look at cutting that number down- that would be way too noisy.
#My initial thought is to see what the top 10 breeds are
#Maybe from there I can either to dummies or aggregate

In [8]:
top_ten = df['Breed'].value_counts()[:11]

In [9]:
top_ten
#bar chart here

MIXED                  29009
LABRADOR RETRIEVER     19713
LAB MIX                17714
GOLDEN RETRIEVER        9344
GER SHEPHERD            8437
SHIH TZU                7976
BEAGLE                  7960
CHIHUAHUA               7664
TAG                     7475
AM PIT BULL TERRIER     7332
YORKSHIRE TERRIER       6268
Name: Breed, dtype: int64

In [10]:
#TAG is not a type of dog. they are denoting that they are putting tags on an existing dog

In [11]:
tag_df = df.loc[df['Breed'] == 'TAG']
tag_df.loc[(tag_df['DogName'] == 'SHADOW') & (tag_df['OwnerZip'] == 15102)]

,LicenseType,Breed,Color,DogName,OwnerZip
ValidDate,,,,,
2015-03-12 11:11:36,Dog Individual Spayed Female,TAG,BLACK,SHADOW,15102
2017-01-10 09:39:46,Dog Individual Spayed Female,TAG,BLACK,SHADOW,15102
2015-12-11 10:35:08,Dog Individual Spayed Female,TAG,BLACK,SHADOW,15102


In [12]:
a = len(tag_df['DogName'])
b = len(tag_df['DogName'].value_counts())
a-b

5708

In [ ]:
#Time to zoom out. What am i hoping to accomplish here? 
#A multivariate time series. OG problem: predict number of licenses dispensed per day. 
#New problem: predict licenses dispenses while considering extra variables. 
#I think top 10 dummies is actually going to benefit me most. 
#Let's start with breed: top 9 and 'other'
## investigate whether an 'other' column is going to unbalance the data too bad; 
#we may need to keep it to top 10 with a disclaimer

In [25]:
df.head()

,LicenseType,Breed,Color,DogName,OwnerZip
ValidDate,,,,,
2014-12-02 09:40:53,Dog Individual Neutered Male,COCKAPOO,BROWN,CHARLEY,15236
2014-12-02 09:45:25,Dog Senior Citizen or Disability Neutered Male,GER SHEPHERD,BLACK/BROWN,TACODA,15238
2014-12-02 09:47:55,Dog Individual Spayed Female,GER SHEPHERD,BLACK,CHARLY,15205
2014-12-02 10:02:33,Dog Individual Spayed Female,LABRADOR RETRIEVER,BLACK,ABBEY,15143
2014-12-02 10:05:50,Dog Individual Female,GER SHORTHAIR POINT,BROWN,CHARLEY,15228


In [26]:
df.isnull().sum()

LicenseType    0
Breed          0
Color          0
DogName        0
OwnerZip       0
dtype: int64

In [47]:
breed_df = pd.DataFrame(df.Breed)
breed_df.set_index(breed_df.index, inplace = True)
breed_df.index = pd.to_datetime(breed_df.index)

In [48]:
grouped_breed = breed_df.resample('B').sum()

In [49]:
grouped_breed.head()

,Breed
ValidDate,
2014-12-02,COCKAPOOGER SHEPHERDGER SHEPHERDLABRADOR RETRI...
2014-12-03,POMERANIAN MIXBISHON FRISE MIXBEAGLE MIXMIXEDB...
2014-12-04,MIXEDOTHERLABRADOR RETRIEVERLABRADOR RETRIEVER...
2014-12-05,LAB MIXCOCKAPOOMORKIEBOXERBOXERBOXERSHIH TZUMI...
2014-12-08,SHETLAND SHEEPDOGPEEKAPOOFOX TERRIER WIREENG S...


In [ ]:
#this is hysterical. i think i'll call that a stopping point for tonight
#next step: get dummies on top 10